In [13]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
import os
prefix = '/content/gdrive/My Drive/'
# modify "customized_path_to_your_homework" here to where you uploaded your homework
customized_path_to_your_homework = 'IDLSProject-main'
sys_path = os.path.join(prefix, customized_path_to_your_homework)
sys.path.append(sys_path)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [14]:
%cd '/content/gdrive/My Drive/IDLSProject-main'

/content/gdrive/My Drive/IDLSProject-main


In [15]:
data_dir = './data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [16]:
import torch
import torch.utils.data
import pandas as pd

# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None)

# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

#### Test data

# Read in only the first 250 rows
test_sample = pd.read_csv(os.path.join(data_dir, 'test.csv'), header=None, names=None)

# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test_sample[[0]].values).float().squeeze()
test_X = torch.from_numpy(test_sample.drop([0], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)

In [17]:

import torch.nn as nn

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self,trial,vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()
        dropout = trial.suggest_uniform("dropout",0.1, 0.6)
        hidden_dim = trial.suggest_int("hidden_dim",16,256)
        embedding_dim = trial.suggest_int("embedding_dim",16,128)

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig(out.squeeze())

In [18]:
epochs = 10

In [19]:
!pip install optuna

     |████████████████████████████████| 308 kB 4.9 MB/s 
     |████████████████████████████████| 210 kB 68.3 MB/s 
     |████████████████████████████████| 81 kB 12.9 MB/s 
     |████████████████████████████████| 78 kB 9.2 MB/s 
     |████████████████████████████████| 49 kB 6.0 MB/s 
     |████████████████████████████████| 112 kB 67.4 MB/s 
     |████████████████████████████████| 146 kB 54.3 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=975327c02eaeccaec1c76be2ae072830551b9cd79de05a2a672f24c92e8d3e0b
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [24]:
import optuna
from optuna.trial import TrialState
import torch.optim as optim

# Define an objective function to be minimized.
def objective(trial):
  optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
  #epochs = trial.suggest_int("epochs",5,15)
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-0)
  #momentum = trial.suggest_uniform("momentum", 0.0, 1.0)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = LSTMClassifier(trial,5000).to(device)
  optimizer = optim.Adam(model.parameters())
  loss_fn = torch.nn.BCELoss()
  for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        hidden = None
        for batch in train_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            optimizer.zero_grad()
            prediction = model(batch_X)
            loss = loss_fn(prediction, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_dl)))
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
          for batch in test_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            prediction = model(batch_X)
            result = np.round(prediction.cpu())
#             results.extend(list(result.numpy()))
#             labels.extend(list(batch_y.numpy()))
            total += batch_y.size(0)
            correct += (result == batch_y).sum().item()
        print("Accuracy:", correct/total)
        accuracy = correct/total
  return accuracy

In [25]:
study = optuna.create_study(direction="maximize",pruner=optuna.pruners.HyperbandPruner(
        min_resource=1, max_resource=epochs, reduction_factor=3
    ),)
study.optimize(objective, n_trials=1)
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

[I 2022-05-14 03:30:17,593] A new study created in memory with name: no-name-8425937f-fae6-4fc5-b48b-249eb8fd7b9a


Epoch: 1, BCELoss: 0.537729003727436
Accuracy: 0.8084
Epoch: 2, BCELoss: 0.4268411916792393
Accuracy: 0.81716
Epoch: 3, BCELoss: 0.32878619214892385
Accuracy: 0.85288
Epoch: 4, BCELoss: 0.27829288572072985
Accuracy: 0.85052
Epoch: 5, BCELoss: 0.24631216710060835
Accuracy: 0.86356
Epoch: 6, BCELoss: 0.21278584172576667
Accuracy: 0.86096
Epoch: 7, BCELoss: 0.18907082679867745
Accuracy: 0.86292
Epoch: 8, BCELoss: 0.17643479952961205
Accuracy: 0.85864
Epoch: 9, BCELoss: 0.14761430918052793
Accuracy: 0.86512
Epoch: 10, BCELoss: 0.1323443582057953


[I 2022-05-14 03:32:17,532] Trial 0 finished with value: 0.86368 and parameters: {'optimizer': 'SGD', 'lr': 0.00044017280457261005, 'dropout': 0.33277629679500353, 'hidden_dim': 86, 'embedding_dim': 69}. Best is trial 0 with value: 0.86368.


Accuracy: 0.86368
Study statistics: 
  Number of finished trials:  1
  Number of complete trials:  1
Best trial:
  Value:  0.86368
  Params: 
    optimizer: SGD
    lr: 0.00044017280457261005
    dropout: 0.33277629679500353
    hidden_dim: 86
    embedding_dim: 69


In [9]:
def test(model, test_dl):
    model.eval()
    correct = 0
    total = 0
#     results = []
#     labels = []
    with torch.no_grad():
        for batch in test_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            prediction = model(batch_X)
            result = np.round(prediction.cpu())
#             results.extend(list(result.numpy()))
#             labels.extend(list(batch_y.numpy()))
            total += batch_y.size(0)
            correct += (result == batch_y).sum().item()
    print("Accuracy:", correct/total)

In [11]:
import numpy as np

In [12]:
test(model, test_dl)

Accuracy: 0.85292
